## Управление историей сообщений в агентах

**Цель**: научить агента автоматически сокращать историю, чтобы:

- Уложиться в лимит токенов модели,
- Удалить технический «мусор» (например, логи инструментов),
- Сохранить самое важное для продолжения диалога.

### Часть 1: Автоматическое суммирование (SummarizationMiddleware)
**Зачем это нужно?**

- Каждое сообщение в истории увеличивает длину промпта.
- У LLM есть максимальный лимит токенов (например, 128K для gpt-4o).
- Если лимит превышен — ошибка или усечение истории → агент «забывает» начало разговора.

**Механизм:**
- Агент получает новые сообщения,
- LangGraph считает токены в них,
- Если суммарное количество новых токенов > 100:
    - Берёт все сообщения, кроме последних 1,
    - Отправляет их в llm с промптом: «Суммируй этот диалог кратко»,
    - Заменяет старые сообщения на одно суммаризованное AIMessage.

In [27]:
from langchain.tools import tool
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [28]:
agent = create_agent(
    model=llm,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=llm,
            max_tokens_before_summary=100,  # ← суммировать, если история > 100 токенов
            messages_to_keep=1              # ← оставить 1 последнее сообщение "как есть"
        )
    ]
)

In [29]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nThe capital of the Moon is Lunapolis, where the weather features clear skies with temperatures ranging from -100°C to 120°C. The city is home to 100,000 cheese miners who are expected to strike due to dissatisfaction with the new president.', additional_kwargs={}, response_metadata={}, id='0397a478-475f-4806-b462-0b34cd718c44'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='550bb531-1c6d-497f-bd33-66f9a6fccc78'),
              AIMessage(content='As the new President of Lunapolis, I would recognize that a strike by 100,000 cheese miners—the backbone of our lunar economy—would be catastrophic. Given the extreme environmental conditions (swinging from -100°C to 120°C), the miners are likely facing immense physical strain and safety concerns.\n\nHere is my four-step plan to addre

In [30]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the Moon is Lunapolis, where the weather features clear skies with temperatures ranging from -100°C to 120°C. The city is home to 100,000 cheese miners who are expected to strike due to dissatisfaction with the new president.


### Часть 2: Удаление сообщений (RemoveMessage)

In [31]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [32]:
agent = create_agent(
    model=llm,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [33]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='ae908605-e610-431e-9ee1-c4c125bc7b9e'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='40acd8c0-d2ef-48c6-95d7-b89e23182b50'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='ae8c9031-2e1b-43d9-a6e5-2fa1b2792ca1'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='3bee443a-8e3c-4c0c-b587-8c780384cd40'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='dec9e518-7a8e-4ef1-be0d-76bdd625f558'),
              AIMessage(content='I am an AI, so I don\'t have a physical body or a temperature in the way a device does. However, if you are asking about **your device**, 

In [34]:
print(response["messages"][-1].content)

I am an AI, so I don't have a physical body or a temperature in the way a device does. However, if you are asking about **your device**, its temperature is a very important clue.

**If your device feels hot to the touch:**
*   **Unplug it immediately.** Overheating can prevent a device from turning on as a safety measure.
*   **Let it cool down** for at least 30 minutes in a cool, dry place.
*   **Check the vents:** Ensure they aren't blocked by dust or lint.

**If your device feels cold (room temperature) and won't turn on:**
1.  **Try a different outlet:** The wall socket might be dead.
2.  **Check the power cable:** Look for frays, bends, or damage. If possible, try a different cable or power brick.
3.  **Perform a "Hard Reset":** 
    *   **For Laptops:** Unplug it, hold the power button down for 30 full seconds, then plug it back in and try to start it.
    *   **For Phones:** Hold the Power button and Volume Down button simultaneously for 10–15 seconds.

**What kind of device is 